## 자연어 생성

* 문자 단위 순환 신경망으로 Latex 등의 구조화된 문서 생성

* 한글 자연어 생성을 위해 조선왕조실록 데이터 파일 다운로드

In [2]:
import tensorflow as tf


# 조선왕조실록 데이터 파일 다운로드
path_to_file = tf.keras.utils.get_file('input.txt', 'http://bit.ly/2Mc3SOV')
# 데이터를 메모리에 불러옵니다. encoding 형식으로 utf-8 을 지정해야합니다. 
train_text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# 텍스트가 총 몇 자인지 확인합니다. 
print('Length of text: {} characters'.format(len(train_text)))
print()
# 처음 100 자를 확인해봅니다. 
print(train_text[:100])

62012502/62012502 [==============================] - 0s 0us/step
Length of text: 26265493 characters

﻿태조 이성계 선대의 가계. 목조 이안사가 전주에서 삼척·의주를 거쳐 알동에 정착하다 
태조 강헌 지인 계운 성문 신무 대왕(太祖康獻至仁啓運聖文神武大王)의 성은 이씨(李氏)요, 휘


In [5]:
import re

def clean_str(string):
  string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\']", " ", string)
  string = re.sub(r"\'ll", " \'ll", string)
  string = re.sub(r",", " , ", string)
  string = re.sub(r"!", " ! ", string)
  string = re.sub(r"\(", "", string)
  string = re.sub(r"\)", "", string)
  string = re.sub(r"\?", " \? ", string)
  string = re.sub(r"\s{2,}", " ", string)
  string = re.sub(r"\'{2,}", "\'", string)
  string = re.sub(r"\'", "", string)
  return string

In [6]:
train_text = train_text.split('\n')
train_text = [clean_str(sentence) for sentence in train_text]
train_text_X = []
for sentence in train_text:
  train_text_X.extend(sentence.split(' '))
  train_text_X.append('\n')
train_text_X = [word for word in train_text_X if word != '']
print(train_text_X[:20])

['태조', '이성계', '선대의', '가계', '목조', '이안사가', '전주에서', '삼척', '의주를', '거쳐', '알동에', '정착하다', '\n', '태조', '강헌', '지인', '계운', '성문', '신무', '대왕']


In [10]:
import numpy as np
# 단어 토큰화
# 단어의 set을 만듭니다.
vocab = sorted(set(train_text_X))
vocab.append('UNK')
print ('{} unique words'.format(len(vocab)))
# vocab list를 숫자로 맵핑하고, 반대도 실행합니다.
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)
text_as_int = np.array([word2idx[c] for c in train_text_X])
# word2idx 의 일부를 알아보기 쉽게 print 해봅니다. print('{')
for word,_ in zip(word2idx, range(10)):
  print(' {:4s}: {:3d},'.format(repr(word), word2idx[word]))
print(' ...\n}')
print('index of UNK: {}'.format(word2idx['UNK']))

332640 unique words
 '\n':   0,
 '!' :   1,
 ',' :   2,
 '000명으로':   3,
 '001':   4,
 '002':   5,
 '003':   6,
 '004':   7,
 '005':   8,
 '006':   9,
 ...
}
index of UNK: 332639


In [14]:
# 기본 데이터셋 만들기
seq_length = 25
examples_per_epoch = len(text_as_int) // seq_length
sentence_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sentence_dataset = sentence_dataset.batch(seq_length+1, drop_remainder=True)
for item in sentence_dataset.take(1):
  print(idx2word[item.numpy()])
  print(item.numpy())

['태조' '이성계' '선대의' '가계' '목조' '이안사가' '전주에서' '삼척' '의주를' '거쳐' '알동에' '정착하다'
 '\n' '태조' '강헌' '지인' '계운' '성문' '신무' '대왕' '의' '성은' '이씨' '요' ',' '휘']
[299305 229634 161443  17430 111029 230292 251081 155087 225462  29027
 190295 256129      0 299305  25624 273553  36147 163996 180466  84413
 224182 164549 230248 210912      2 330313]


In [15]:
def split_input_target(chunk):
  return [chunk[:-1], chunk[-1]]
train_dataset = sentence_dataset.map(split_input_target)
for x,y in train_dataset.take(1):
  print(idx2word[x.numpy()])
  print(x.numpy())
  print(idx2word[y.numpy()])
  print(y.numpy())

['태조' '이성계' '선대의' '가계' '목조' '이안사가' '전주에서' '삼척' '의주를' '거쳐' '알동에' '정착하다'
 '\n' '태조' '강헌' '지인' '계운' '성문' '신무' '대왕' '의' '성은' '이씨' '요' ',']
[299305 229634 161443  17430 111029 230292 251081 155087 225462  29027
 190295 256129      0 299305  25624 273553  36147 163996 180466  84413
 224182 164549 230248 210912      2]
휘
330313


In [16]:
# 데이터셋 shuffle, batch 설정
BATCH_SIZE = 512
steps_per_epoch = examples_per_epoch // BATCH_SIZE
BUFFER_SIZE = 10000
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [17]:
# 단어 단위 생성 모델 정의
total_words = len(vocab)
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(total_words, 100, input_length=seq_length),
  tf.keras.layers.LSTM(units=100, return_sequences=True),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.LSTM(units=100),
  tf.keras.layers.Dense(total_words, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 100)           33264000  
                                                                 
 lstm (LSTM)                 (None, 25, 100)           80400     
                                                                 
 dropout (Dropout)           (None, 25, 100)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 332640)            33596640  
                                                                 
Total params: 67,021,440
Trainable params: 67,021,440
Non-trainable params: 0
_________________________________________________________________


In [19]:
# 단어 단위 생성 모델 학습
from tensorflow.keras.preprocessing.sequence import pad_sequences

def testmodel(epoch, logs):
  if epoch % 5 != 0 and epoch != 49:
    return
  test_sentence = train_text[0]

  next_words = 100
  for _ in range(next_words):
    test_text_X = test_sentence.split(' ')[-seq_length:]
    test_text_X = np.array([word2idx[c] if c in word2idx
                            
  else word2idx['UNK'] for c in test_text_X])
    test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding='pre', value=word2idx['UNK'])
  

  output_idx = np.argmax(model.predict(test_text_X),axis=-1)
  test_sentence += ' ' + idx2word[output_idx[0]]
  print()
  print(test_sentence)
  print()

testmodelcb = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)
history = model.fit(train_dataset.repeat(), epochs=3, steps_per_epoch=steps_per_epoch, callbacks=[testmodelcb], verbose=2)

Epoch 1/3
1/1 [==============================] - 1s 1s/step

 태조 이성계 선대의 가계 목조 이안사가 전주에서 삼척 의주를 거쳐 알동에 정착하다  


533/533 - 177s - loss: 8.5077 - accuracy: 0.0735 - 177s/epoch - 332ms/step
Epoch 2/3
533/533 - 158s - loss: 8.1816 - accuracy: 0.0800 - 158s/epoch - 297ms/step
Epoch 3/3
533/533 - 147s - loss: 7.9089 - accuracy: 0.0878 - 147s/epoch - 276ms/step
